In [11]:
!pip install --upgrade py2neo
!pip install ipykernel
!pip install --upgrade neo4jupyter
!pip install --upgrade neo4j
!pip install pandas
!pip install kagglehub


import os
import pandas as pd
from tqdm import tqdm

#permite conectarse a la BD
#Fuente: https://py2neo.org/2021.1/
import py2neo
from py2neo import Graph,Node,Relationship

#permite representar gráficamente los grafos obtenidos
#Fuente: https://github.com/merqurio/neo4jupyter
import neo4jupyter
neo4jupyter.init_notebook_mode()


print ('py2neo version', py2neo.__version__)



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


<IPython.core.display.Javascript object>

py2neo version 2021.2.4


## Autenticación a la base ##

In [12]:
# url de la BD
dburl='bolt://localhost:7687'

#indica usuario y pass a la BD
user="neo4j"
pasw="Sustiuir por la contraseña de la BD"

graph = Graph(dburl, auth=(user, pasw))

from pandas import read_csv,isnull
from neo4j import GraphDatabase

uri             = dburl # url of your neo4j proyect, for example "bolt://localhost:7687"

userName        = user # username

password        = pasw # user password

# Connect to the neo4j database server

graphDB_Driver  = GraphDatabase.driver(uri, auth=(userName, password))

## Importación de datos

In [13]:
file_noc = 'files/teams.csv'

# LOAD TEAMS + ARENA

with graphDB_Driver.session() as graphDB_Session:
    df = read_csv(file_noc,low_memory=False)
    largo = len(df.index)
    print(largo)
    for idx in tqdm(range(0,largo)):
        # get the information of each row
        wanted_df_slice = df.iloc[[idx]]
        team_id = str(wanted_df_slice.TEAM_ID.values[0])
        name = str(wanted_df_slice.NICKNAME.values[0])
        arena = str(wanted_df_slice.ARENA.values[0])
        # cypher query to create the players nodes and their relationship "PLAYED" that matches each player with the games they played and their stats
        cqlNodeQuery  = 'MERGE (t: Team {id: \'' + team_id +'\', name: \'' + name +'\' })'
        cqlNodeQuery += 'MERGE (a: Arena {id: \''+ arena +'\'})'
        cqlNodeQuery += 'MERGE (t)-[:HAS_ARENA]->(a)'

        # Query the graph
        nodes = graphDB_Session.run(cqlNodeQuery)

30


100%|██████████| 30/30 [00:02<00:00, 11.17it/s]


In [14]:
file_noc = 'files/ranking.csv'

# LOAD CONFERENCES and relationships

with graphDB_Driver.session() as graphDB_Session:
    df = read_csv(file_noc,low_memory=False)
    largo = len(df.index)
    teams=set()
    for idx in range(0,largo):
        # get the information of each row
        wanted_df_slice = df.iloc[[idx]]
        team_id = str(wanted_df_slice.TEAM_ID.values[0])
        conference = str(wanted_df_slice.CONFERENCE.values[0])
        # cypher query to create the players nodes and their relationship "PLAYED" that matches each player with the games they played and their stats
        cqlNodeQuery = 'MATCH (t:Team {id: \''+ team_id +'\'}) '
        cqlNodeQuery += 'MERGE (conf:Conference {id: \''+ conference +'\'})'
        cqlNodeQuery += 'MERGE (t)-[:BELONGS]->(conf)'
        if len(teams) < 30 :
          if team_id not in teams: # do not add team to conference twice
            nodes = graphDB_Session.run(cqlNodeQuery)
            teams.add(team_id)
        else:
          break;


In [15]:
file_noc = 'files/games.csv'

with graphDB_Driver.session() as graphDB_Session:
    df = read_csv(file_noc,low_memory=False)
    largo = len(df.index) #
    with graphDB_Session.begin_transaction() as tx:
      for idx in tqdm(range(0,largo)):
          # get the information of each row
          wanted_df_slice = df.iloc[[idx]]

          game_id = str(wanted_df_slice.GAME_ID.values[0])
          home_team_wins = bool(wanted_df_slice.HOME_TEAM_WINS.values[0])
          home_team = str(wanted_df_slice.HOME_TEAM_ID.values[0])
          away_team = str(wanted_df_slice.VISITOR_TEAM_ID.values[0])
          winner_team = home_team if home_team_wins else away_team
          loser_team = away_team if home_team_wins else home_team
          season = str(wanted_df_slice.SEASON.values[0])
          date = str(wanted_df_slice.GAME_DATE_EST.values[0])
          #home team data

          home_pts = "0" if isnull(wanted_df_slice.PTS_home.values[0]) else str(wanted_df_slice.PTS_home.values[0])
          home_fg_pct = "0" if isnull(wanted_df_slice.FG_PCT_home.values[0]) else str(wanted_df_slice.FG_PCT_home.values[0])
          home_ft_pct = "0" if isnull(wanted_df_slice.FT_PCT_home.values[0]) else str(wanted_df_slice.FT_PCT_home.values[0])
          home_fg3m_pct = "0" if isnull(wanted_df_slice.FG3_PCT_home.values[0]) else str(wanted_df_slice.FG3_PCT_home.values[0])
          home_reb = "0" if isnull(wanted_df_slice.REB_home.values[0]) else str(wanted_df_slice.REB_home.values[0])
          home_ast = "0" if isnull(wanted_df_slice.AST_home.values[0]) else str(wanted_df_slice.AST_home.values[0])

          #away team data

          away_pts = "0" if isnull(wanted_df_slice.PTS_away.values[0]) else str(wanted_df_slice.PTS_away.values[0])
          away_fg_pct = "0" if isnull(wanted_df_slice.FG_PCT_away.values[0]) else str(wanted_df_slice.FG_PCT_away.values[0])
          away_ft_pct = "0" if isnull(wanted_df_slice.FT_PCT_away.values[0]) else str(wanted_df_slice.FT_PCT_away.values[0])
          away_fg3m_pct = "0" if isnull(wanted_df_slice.FG3_PCT_away.values[0]) else str(wanted_df_slice.FG3_PCT_away.values[0])
          away_reb = "0" if isnull(wanted_df_slice.REB_away.values[0]) else str(wanted_df_slice.REB_away.values[0])
          away_ast = "0" if isnull(wanted_df_slice.AST_away.values[0]) else str(wanted_df_slice.AST_away.values[0])

        # cypher query to create the relationship "HOME_TEAM" that matches the local team stats of each game
          full_query  = 'MATCH (ht:Team {id: \''+ home_team +'\'}) MATCH (at:Team {id: \''+ away_team+'\'})'
          full_query  += ' MERGE (g: Game {id: \'' + game_id +'\', date: DATE(\'' + date+'\')}) MERGE (tsh: TeamSeason {id:\''+home_team+season+'\'}) MERGE (ht)-[:_'+season+']->(tsh)'
          full_query  += ' MERGE (g)-[r:HOME_TEAM {PTS: '+ home_pts +', FG_PCT: '+ home_fg_pct +', FT_PCT: '+home_ft_pct+', AST: '+home_ast+', REB: '+home_reb+' }]->(tsh)'
          full_query += ' MERGE (s:Season {id: \'' + season +'\'}) MERGE (tsh)-[:SEASON]->(s)'
          # cypher query to create the relationship "AWAY_TEAM" that matches the away team stats of each game
          full_query += ' MERGE (tsa: TeamSeason {id:\''+away_team+season+'\'}) MERGE (at)-[:_'+season+']->(tsa)'
          full_query += ' MERGE (g)-[o:AWAY_TEAM {PTS: '+ away_pts +', FG_PCT: '+ away_fg_pct +', FT_PCT: '+away_ft_pct+', AST: '+away_ast+', REB: '+away_reb+' }]->(tsa)'
          full_query += ' MERGE (tsa)-[:SEASON]->(s)'
          # add winner and loser
          full_query +=' MERGE (g)-[:WINNER]->(tsh) MERGE (g)-[:LOSER]->(tsa)' if home_team_wins else ' MERGE (g)-[:WINNER]->(tsa) MERGE (g)-[:LOSER]->(tsh)'


          # Season query
          tx.run(full_query)
          if idx % 100 == 0:
             tx.commit()
             tx = graphDB_Session.begin_transaction()

      tx.commit()


100%|██████████| 26651/26651 [06:45<00:00, 65.66it/s]


In [16]:
file_noc = 'files/games_details.csv'
time_dict = {}
with graphDB_Driver.session() as graphDB_Session:
    df = read_csv(file_noc,low_memory=False)
    largo = len(df.index)
    print(largo)

    with graphDB_Session.begin_transaction() as tx:
      # 68501
      for idx in tqdm(range(268501,largo)): # Re-run from new range if fails
          # get the information of each row
          wanted_df_slice = df.iloc[[idx]]
          game_id = str(wanted_df_slice.GAME_ID.values[0])
          name = '\"'+str(wanted_df_slice.PLAYER_NAME.values[0])+'\"'
          player_id = str(wanted_df_slice.PLAYER_ID.values[0])
          team_id = str(wanted_df_slice.TEAM_ID.values[0])
          time_played = str(wanted_df_slice.MIN.values[0])
          secs = 0 # defalut value that represents "Do not play"
          # if Min column has some value, we need to calculate the seconds and store it
          if not isnull(wanted_df_slice.MIN.values[0]) :
              # we split in minutes and seconds
              mins_slice = time_played.split(":")
              # if play at least 1 minute, we need to calculate that time in seconds
              if len(mins_slice) > 1 : # there is wrong data of "-5" seconds played by some players
                  secs = int(float(mins_slice[0]))*60+int(float(mins_slice[1]))

          fgm = "0" if isnull(wanted_df_slice.FGM.values[0]) else str(wanted_df_slice.FGM.values[0])
          fga = "0" if isnull(wanted_df_slice.FGA.values[0]) else str(wanted_df_slice.FGA.values[0])
          fg3a = "0" if isnull(wanted_df_slice.FG3A.values[0]) else str(wanted_df_slice.FG3A.values[0])
          fg3m = "0" if isnull(wanted_df_slice.FG3M.values[0]) else str(wanted_df_slice.FG3M.values[0])
          ftm = "0" if isnull(wanted_df_slice.FTM.values[0]) else str(wanted_df_slice.FTM.values[0])
          fta = "0" if isnull(wanted_df_slice.FTA.values[0]) else str(wanted_df_slice.FTA.values[0])
          oreb = "0" if isnull(wanted_df_slice.OREB.values[0]) else str(wanted_df_slice.OREB.values[0])
          dreb = "0" if isnull(wanted_df_slice.DREB.values[0]) else str(wanted_df_slice.DREB.values[0])
          reb = "0" if isnull(wanted_df_slice.REB.values[0]) else str(wanted_df_slice.REB.values[0])
          ast = "0" if isnull(wanted_df_slice.AST.values[0]) else str(wanted_df_slice.AST.values[0])
          stl = "0" if isnull(wanted_df_slice.STL.values[0]) else str(wanted_df_slice.STL.values[0])
          blk = "0" if isnull(wanted_df_slice.BLK.values[0]) else str(wanted_df_slice.BLK.values[0])
          to = "0" if isnull(wanted_df_slice.TO.values[0]) else str(wanted_df_slice.TO.values[0])
          pf = "0" if isnull(wanted_df_slice.PF.values[0]) else str(wanted_df_slice.PF.values[0])
          pts = "0" if isnull(wanted_df_slice.PTS.values[0]) else str(wanted_df_slice.PTS.values[0])
          # cypher query to create the players nodes and their relationship "PLAYED" that matches each player with the games they played and their stats
          cqlNodeQuery  = 'MATCH (g: Game {id: \'' + game_id +'\' })'
          cqlNodeQuery += 'MERGE (p: Player {id: \''+ player_id +'\', name: '+ name +'})'

          cqlNodeQuery += 'MERGE (p)-[r:PLAYED {team: \''+ team_id +'\', time_played:'+ str(secs) +', FGM: '+fgm+', FGA: '+fga+', FG3A: '+fg3a+',FG3M: '+fg3m+',FTM: '+ftm+', FTA: '+fta+', OREB: '+oreb+',DREB: '+dreb+', REB: '+reb+', AST: '+ast+',STL: '+stl+', BLK: '+blk+', FOULS: '+to+', PF: '+pf+', PTS: '+pts+' }]->(g)'
          cqlNodeQuery +=' SET g.time_played = COALESCE(g.time_played, 0) +'+str(secs/10)


          # Query the graph
          tx.run(cqlNodeQuery)
          if idx % 100 == 0:
             tx.commit()
             last_loaded_row = idx
             tx = graphDB_Session.begin_transaction()

      tx.commit()


668628


100%|██████████| 400127/400127 [1:24:07<00:00, 79.28it/s]  


## Creación de aristas para el subgrafo de distancias

In [ ]:
with graphDB_Driver.session() as session:
  session.run("""
    MATCH (p1:Player)-[pl1:PLAYED]->(g:Game)<-[pl2:PLAYED]-(p2:Player)
    WHERE elementId(p1) < elementId(p2)
    WITH p1, p2, count(g) AS shared_games
    MERGE (p1)-[r:SHARED_GAME_WITH]->(p2)
    SET r.weight = 1.0/shared_games, r.shared_games = shared_games
  """)


  session.run("""
  MATCH (p1:Player)-[pl1:PLAYED]->(g:Game)<-[pl2:PLAYED]-(p2:Player)
  WHERE elementId(p1) < elementId(p2) AND pl1.team = pl2.team
  WITH p1, p2, count(g) AS shared_games_as_ally
  MERGE (p1)-[r:ALLY]->(p2)
  SET r.weight = 1.0/shared_games_as_ally, r.shared_games_as_ally = shared_games_as_ally
  """)

  session.run("""
    MATCH (p1:Player)-[pl1:PLAYED]->(g:Game)<-[pl2:PLAYED]-(p2:Player)
    WHERE elementId(p1) < elementId(p2) AND pl1.team <> pl2.team
    WITH p1, p2, count(g) AS shared_games_as_rival
    MERGE (p1)-[r:RIVAL]->(p2)
    SET r.weight = 1.0/shared_games_as_rival, r.shared_games_as_rival = shared_games_as_rival
  """)

## Ejemplo de consulta para distancia de jugadores:

In [19]:
with graphDB_Driver.session() as session:
    session.run("""
        MATCH p = SHORTEST 1 (p1:Player {name:"LeBron James"})-[:SHARED_GAME_WITH]-+(p2:Player {name: "Kyrie Irving"})
        RETURN p
    """)

## Distancia General

Creación del grafo proyectado:

In [20]:
with graphDB_Driver.session() as session:
    session.run("""
        CALL gds.graph.project(
        'distanceGraph',
        'Player',
        {
                        SHARED_GAME_WITH: {
                                    orientation: 'UNDIRECTED',
                                    properties: 'weight'
                                }
                    }
        )
        YIELD graphName, nodeCount, relationshipCount;
    """)

Betweenness Centrality:

In [27]:
with graphDB_Driver.session() as session:
    result = session.run("""
        CALL gds.betweenness.stream('distanceGraph', { relationshipWeightProperty: 'weight' })
        YIELD nodeId, score
        RETURN gds.util.asNode(nodeId).name AS name, score
        ORDER BY score DESC
        LIMIT 10;
    """)

    for record in result:
        print(f"{record['name']}: {record['score']:.4f}")


LeBron James: 74949.8788
Jamal Crawford: 72305.6500
Kyle Korver: 66967.0833
Jeff Teague: 62555.3278
LaMarcus Aldridge: 61787.7634
Dirk Nowitzki: 61424.1357
Udonis Haslem: 60960.3268
James Harden: 58319.2837
Marvin Williams: 57050.1326
JJ Redick: 56404.3742


Degree Centrality:

In [28]:
with graphDB_Driver.session() as session:
    result = session.run("""
        CALL gds.degree.stream('distanceGraph', { relationshipWeightProperty: 'weight' })
        YIELD nodeId, score
        RETURN gds.util.asNode(nodeId).name AS name, score
        ORDER BY score DESC
        LIMIT 10;
    """)

    for record in result:
        print(f"{record['name']}: {record['score']:.4f}")

Udonis Haslem: 487.7100
Kyle Korver: 483.9077
Tyson Chandler: 474.4265
Dwight Howard: 469.1686
Dwyane Wade: 458.8847
Pau Gasol: 457.9605
Vince Carter: 454.6061
Trevor Ariza: 442.5471
Dirk Nowitzki: 439.6372
Joe Johnson: 432.1212


Closeness Centrality:

In [29]:
with graphDB_Driver.session() as session:
    result = session.run("""
        CALL gds.closeness.stream('distanceGraph')
        YIELD nodeId, score
        RETURN gds.util.asNode(nodeId).name AS name, score
        ORDER BY score DESC
        LIMIT 10;
    """)

    for record in result:
        print(f"{record['name']}: {record['score']:.4f}")

Udonis Haslem: 0.7768
Kyle Korver: 0.7751
Vince Carter: 0.7736
LeBron James: 0.7603
Tyson Chandler: 0.7595
Pau Gasol: 0.7590
Dwyane Wade: 0.7562
Zaza Pachulia: 0.7541
Nene: 0.7541
Dirk Nowitzki: 0.7536


Detección de comunidades: Louvain

In [30]:
with graphDB_Driver.session() as session:
    result = session.run("""
        CALL gds.louvain.stats('distanceGraph', {relationshipWeightProperty: 'weight'})
        YIELD communityCount, modularity
        RETURN communityCount, modularity;
    """)

    for record in result:
        print(f"Community Count: {record['communityCount']}, Modularity: {record['modularity']:.4f}")

Community Count: 3, Modularity: 0.3116


Detección de comunidades: Leiden

In [31]:
with graphDB_Driver.session() as session:
    result = session.run("""
        CALL gds.leiden.stats('distanceGraph', {relationshipWeightProperty: 'weight'})
        YIELD communityCount, modularity
        RETURN communityCount, modularity;
    """)

    for record in result:
        print(f"Community Count: {record['communityCount']}, Modularity: {record['modularity']:.4f}")

Community Count: 3, Modularity: 0.3116


## DISTANCIA ALIADOS

Creación del grafo proyectado:

In [47]:
with graphDB_Driver.session() as session:
    session.run("""
        CALL gds.graph.project(
        'allyGraph',
        'Player',
        {
                        ALLY: {
                                    orientation: 'UNDIRECTED',
                                    properties: 'weight'
                                }
                    }
        )
        YIELD graphName, nodeCount, relationshipCount;
    """)

Detección de comunidades: Louvain

In [51]:
with graphDB_Driver.session() as session:
    result = session.run("""
        CALL gds.louvain.stats('allyGraph',{ relationshipWeightProperty: 'weight' })
        YIELD communityCount, modularity
        RETURN communityCount, modularity;
    """)

    for record in result:
        print(f"Community Count: {record['communityCount']}, Modularity: {record['modularity']:.4f}")

Community Count: 19, Modularity: 0.4319


Detección de comunidades: Leiden

In [52]:
with graphDB_Driver.session() as session:
    result = session.run("""
        CALL gds.leiden.stats('allyGraph', { relationshipWeightProperty: 'weight' })
        YIELD communityCount, modularity
        RETURN communityCount, modularity;
    """)

    for record in result:
        print(f"Community Count: {record['communityCount']}, Modularity: {record['modularity']:.4f}")

Community Count: 20, Modularity: 0.4306


## Distancia Rival

Creación del grafo proyectado

In [38]:
with graphDB_Driver.session() as session:
    session.run("""
        CALL gds.graph.project(
        'rivalGraph',
        'Player',
        {
                    RIVAL: {
                        orientation: 'UNDIRECTED',
                        properties: 'weight'
                    }
                    }
        )
        YIELD graphName, nodeCount, relationshipCount;
    """)

Detección de comunidades: Louvain

In [39]:
with graphDB_Driver.session() as session:
    result = session.run("""
        CALL gds.louvain.stats('rivalGraph',{ relationshipWeightProperty: 'weight' })
        YIELD communityCount, modularity
        RETURN communityCount, modularity;
    """)

    for record in result:
        print(f"Community Count: {record['communityCount']}, Modularity: {record['modularity']:.4f}")

Community Count: 3, Modularity: 0.3088


Detección de comunidades: Leiden

In [53]:
with graphDB_Driver.session() as session:
    result = session.run("""
        CALL gds.leiden.stats('rivalGraph', { relationshipWeightProperty: 'weight' })
        YIELD communityCount, modularity
        RETURN communityCount, modularity;
    """)

    for record in result:
        print(f"Community Count: {record['communityCount']}, Modularity: {record['modularity']:.4f}")

Community Count: 3, Modularity: 0.3088
